In [2]:
import ipynb.fs.full.get_news_headlines as gethl
import ipynb.fs.full.get_article_content as getart
import ipynb.fs.full.write_data as gswd
import pandas
import json
import os

In [3]:
def main():
    try:
        hl = gethl.read_livemint_headlines()  #Get headlines
        #outcomes = pandas.DataFrame()
        
        if(not(hl.empty)):
            
            classified_hl = pandas.DataFrame(json.loads(gethl.classify_headlines(hl['title'].tolist())))  # converting to json the model reply from Gemini AI.
            classified_titles = hl.join(classified_hl, lsuffix='_orig', rsuffix='_copy')   #make one df given with surety that matching is happening on indexes because Title matching used to give missed matches.. 
            classified_titles = classified_titles[classified_titles.classification]  #keeping only the titles with a analyst recommerndation here.
            classified_titles.drop_duplicates(subset=['link_id'], keep='first', inplace=True, ignore_index=False)        #remove duplicates that can creep in from multiple uls   
            gswd.write_headlines(classified_titles)  #write titles to a google file

            for idx, row in classified_titles.iterrows():
                article = getart.get_livemint_articles(row['link'])   #read each article and get it title, subheader, body, and date
                table = getart.get_value_table(article)               # send it to to LLM to make sense of it 
                value_table = pandas.DataFrame(json.loads(table))      # Store the outcomes
                if(not(value_table.empty)):
                    value_table.insert(0, 'uuid', str(row['uuid']))    # This UUID object is not serializable so better we convert it to string here
                    #value_table.insert(1, 'link', row['link'])        #Just having the UUID should solve the problem  now.
                    #value_table.insert(2, 'site', 'Livemint')
                    #print(value_table)
                    gswd.write_recommendations(value_table)
                else:
                    print(f'No recommendations received for article: {row['link']}')
                
        else:
            print('No Headlines Received')
    except Exception as e:
        print(e)
    return 0

In [4]:
if __name__ == '__main__':
    hl = main()